All code can be found at https://github.com/frankjoshua/demo-ros-cmdvel-jupyter

# First import the ros python libraries

In [2]:
import rospy as rp

In [3]:
import rosnode
rosnode.get_node_names()

['/rosout']

# Create a node in order to send and recieve ROS messages

In [4]:
rp.init_node('jupyter_demo')
rosnode.get_node_names()

['/rosout', '/jupyter_demo']

In [ ]:
topicInfo = rosnode.get_node_info_description('/turtlesim')
print(topicInfo)

#### ** Start Turtle sim **
# List available topics

In [10]:
rp.get_published_topics() # Does not show unsubscribed topics

[['/turtle1/color_sensor', 'turtlesim/Color'],
 ['/rosout', 'rosgraph_msgs/Log'],
 ['/rosout_agg', 'rosgraph_msgs/Log'],
 ['/turtle1/pose', 'turtlesim/Pose']]

# Move the turtle by sending messages to the '/turtle1/cmd_vel' topic

In [8]:
import rostopic as rt
topicInfo = rt.get_info_text('/turtle1/cmd_vel')
print(topicInfo)

Type: geometry_msgs/Twist

Publishers: None

Subscribers: 
 * /turtlesim (http://127.0.0.1:41107/)




# The '/turtle1/cmd_vel' topic is a Twist type

In [11]:
topicType = rt.get_topic_type('/turtle1/cmd_vel')
print(topicType)

('geometry_msgs/Twist', '/turtle1/cmd_vel', None)


In [12]:
import rosmsg
msgInfo = rosmsg.get_msg_text(topicType[0])
print(msgInfo)

geometry_msgs/Vector3 linear
  float64 x
  float64 y
  float64 z
geometry_msgs/Vector3 angular
  float64 x
  float64 y
  float64 z



[WARN] [1578021900.005735]: Failed to load Python extension for LZ4 support. LZ4 compression will not be available.


# Twist message consists of 2 Vector3 messages. <br> Each containing 3 floats. X, Y and Z. <br> One for Linear motion and the second for Angular motion.  

# Linear values are meters per second.<br>Angular values are radians per second.

# Create a publisher to send messages to a topic

In [13]:
from geometry_msgs.msg import Twist
publisher = rp.Publisher('turtle1/cmd_vel', Twist, queue_size=0)

# To move the turtle forward send a Twist message with a positive linear x value to the 'turtle/cmd_vel' topic

In [14]:
twistMessage = Twist()
twistMessage.linear.x = 5.0

publisher.publish(twistMessage)

### The turtle will stop moving if no message is received for over one second. This behavior varies from robot to robot.
# Negative values will move the turtle backwards

In [15]:
twistMessage.linear.x = -5.0
publisher.publish(twistMessage)


# Rotate counter clockwise by sending Twist message with angular z set to a positive value

In [16]:
twistMessage.linear.x = 5.0
twistMessage.angular.z = 6.2
publisher.publish(twistMessage)

# Clockwise with negative values

In [17]:
twistMessage.angular.z = -3.0
publisher.publish(twistMessage)

In [18]:
publisher.publish(twistMessage)

# Use standard python code to control the robot

In [19]:
vel_msg = Twist()

for n in range(5):
    vel_msg.linear.x = 5
    vel_msg.angular.z = 0
    publisher.publish(vel_msg)
    rp.sleep(0.5)
    for i in range(15):
        vel_msg.linear.x = 4 + n * n
        vel_msg.angular.z = 100 / (i + 10)
        publisher.publish(vel_msg)
        rp.sleep(0.25)

# **Jupyter ROS** can be used to create widgets to send messages

In [ ]:
import jupyros as jr
from geometry_msgs.msg import Twist
jr.publish('/turtle1/cmd_vel', Twist)

### Right click to move the output to a new view

# Echo topic

In [ ]:
def callback(msg):
    print(msg)
jr.subscribe('/turtle1/cmd_vel', Twist, callback)

In [ ]:
import rosservice as rs
rs.get_service_list()

In [ ]:
from std_srvs.srv import Empty
reset = rp.ServiceProxy('/reset', Empty)
reset()

In [ ]:
from IPython.core.display import display, HTML
import ipywidgets as iw
iw.HTML('<img src="https://placekitten.com/g/300/150">')

In [ ]:
from ipywidgets import IntSlider
slider = IntSlider(value=50)
slider

In [ ]:
javascript = '''
<script src="https://cdnjs.cloudflare.com/ajax/libs/nipplejs/0.7.3/nipplejs.min.js"></script>
<script>
    var options = {
        zone: document.getElementById('zone_joystick'),
        color: 'blue',
        mode: 'static',
        position: {top: '50%', left: '50%'}
    };
    var manager = nipplejs.create(options);
    manager.on('move', function(evt, data) {
        debug.innerHTML = JSON.stringify(data.position);
      })
</script>
<div id="zone_joystick" style="height:250px;">
<div id="debug"/>
</div>
'''

display(HTML(javascript))

In [ ]:
from jupyros import ros3d
v = ros3d.Viewer()
rc = ros3d.ROSConnection()
tf_client = ros3d.TFClient(ros=rc, fixed_frame='/slamware_map')
v

In [ ]:
laser_view = ros3d.LaserScan(topic="/slamware_ros_sdk_server_node/scan", ros=rc, tf_client=tf_client)
g = ros3d.GridModel()
v.objects = [g, laser_view]

In [ ]:
laser_view.point_size = 1
laser_view.static_color = "green"

In [ ]:
videoFeed = '<img src="http://192.168.2.84:8090/stream/video.mjpeg" width="60%"" height="60%"/>'
display(HTML(videoFeed))